# Gastric Cancer Detection using resnet_50 and histopathology image patches


*   This notebook entails the creation of a gastric cancer detection model that uses image patches and the resnet_50 pretrained CNN model.
*   The dataset used can be accessed from this site :  https://figshare.com/articles/dataset/GasHisSDB/15066147/1
*   The dataset was divided separately as follows : 70% training, 15% validation, and 15% testing.
*   This code was run on Google Colab using T4 GPU.




1. Import the necessary libraries.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import confusion_matrix, roc_curve, auc

2. Set the device.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

3. Connect to Google Drive to access the zip folders containing the image patches.

In [ ]:
from google.colab import drive
# Mount Google Drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PATH="/content/gdrive/MyDrive/GasHisDB/the_resnet_50_model.pth" #path where the trained model will be saved

4. Define the image transformation pipelines.

In [ ]:
data_transforms = {

    'Train' : transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'Validate': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ])
}

5. Extract the images from the zips.

In [ ]:
# don't run again
#extract dataset
import zipfile
import os

#Abnormal
ZIP_PATH_120 = "/content/gdrive/MyDrive/GasHisDB/120_dataset.zip"
IMAGE_FOLDER_120 = "/content/gdrive/MyDrive/GasHisDB/120_b"

# Ensure that the extraction folder exists
if not os.path.exists(IMAGE_FOLDER_120):
    os.makedirs(IMAGE_FOLDER_120)

# Open the zip file and extract its contents without creating a new folder
with zipfile.ZipFile(ZIP_PATH_120, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        print(f"Extracting: {file_info.filename}")
        zip_ref.extract(file_info, path=IMAGE_FOLDER_120)

In [ ]:
!ls /content/gdrive/MyDrive/GasHisDB/120_b/120_dataset/Train/Normal/Normal-02185.png

6. Create data loaders for training and validation image datasets.

In [ ]:
data_dir = '/content/gdrive/MyDrive/GasHisDB/120_b/120_dataset'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['Train', 'Validate']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=2) for x in ['Train', 'Validate']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['Train', 'Validate']}
class_names = image_datasets['Train'].classes

7. Define the function for training the model.

In [ ]:
### Training the model

def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Train', 'Validate']:
            if phase == 'Train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'Train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'Validate' and epoch_acc > best_acc:

                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), './best-model-checkpoint.pt')
                best_acc = epoch_acc


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

8. Load the resnet_50 pretrained model and train the model while performing validation.

In [ ]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = model_ft.to(device)

#criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

In [ ]:
print("\nSaving the model...")
torch.save(model_ft, PATH)
print("\nThe Model is Saved...Hurray")

In [ ]:
!ls /content/gdrive/MyDrive/GasHisDB/120/120_dataset/Test/Normal/Normal-01006.png

9. Test an image using the model

In [ ]:
#test a single image
from PIL import Image

# Load the saved model checkpoint
model = torch.load(PATH)
model.eval()

# Define the transformations for the test image
data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and preprocess the test image
test_image_path = '/content/gdrive/MyDrive/GasHisDB/120/120_dataset/Test/Normal/Normal-01006.png'  # the path to test image
test_image = Image.open(test_image_path)
input_image = data_transforms(test_image).unsqueeze(0)  # Add a batch dimension
#'input_image' is input tensor and 'device' is device (cuda)
input_image = input_image.to(device)

# Perform inference on the test image
with torch.no_grad():
    outputs = model(input_image)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Softmax to get probabilities
    _, predicted = torch.max(outputs, 1)

# Get the predicted class index and class name
predicted_class_index = predicted.item()
class_names = ['abnormal', 'normal']
predicted_class_name = class_names[predicted_class_index]
predicted_probability = probabilities[0][predicted_class_index].item() * 100  # Probability in percentage

print(f"The predicted class is: {predicted_class_name}")
print(f"The probability of being {predicted_class_name} is: {predicted_probability:.2f}%")


In [ ]:
model = torch.load(PATH)

10. Test the model using the test dataset.

In [ ]:
#FinalTestfolder
ZIP_PATH_120 = "/content/gdrive/MyDrive/GasHisDB/GasHisDB_Final_Test.zip"
IMAGE_FOLDER_120 = "/content/gdrive/MyDrive/GasHisDB/GasHisDB_Final_Test"

# Ensure that the extraction folder exists
if not os.path.exists(IMAGE_FOLDER_120):
    os.makedirs(IMAGE_FOLDER_120)

# Open the zip file and extract its contents without creating a new folder
with zipfile.ZipFile(ZIP_PATH_120, 'r') as zip_ref:
    for file_info in zip_ref.infolist():
        print(f"Extracting: {file_info.filename}")
        zip_ref.extract(file_info, path=IMAGE_FOLDER_120)

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the transformation for test data
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the input size of the model
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize with the same values used in training
])

# Load the test dataset
test_dataset = datasets.ImageFolder(root='/content/gdrive/MyDrive/GasHisDB/GasHisDB_Final_Test', transform=test_transform)

# Create the DataLoader for test dataset
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)


In [ ]:
model = torch.load(PATH)
model = model.to(device)
model.eval()  # Set the model to evaluation mode

true_labels = []
predictions = []
probs = []

with torch.no_grad(): #no need to calculate gradients, only necessary during training when we want to update the weights
    for inputs, labels in test_loader: #yields batches of input data and corresponding labels from the test dataset
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        probabilities = torch.nn.functional.softmax(outputs, dim=1) #softmax function is applied across the second dimension (dim=1), which corresponds to the class probabilities for each input.

        _, predicted = torch.max(outputs, 1)# finds the maximum value in the outputs along dimension 1, which represents the most likely class according to the model, only returns predicted class nt max value
        true_labels.extend(labels.cpu().numpy()) # true labels for the batch are converted from a PyTorch tensor to a NumPy array and added to the true_labels list
        predictions.extend(predicted.cpu().numpy()) # the predicted classes for the batch are converted to a NumPy array and added to the predictions list
        probs.extend(probabilities.cpu().numpy())

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt

# Calculate accuracy, precision, recall, and F1 score
print(classification_report(true_labels, predictions))
print(f"Accuracy: {accuracy_score(true_labels, predictions)}")

# Calculate the ROC-AUC score and plot the ROC curve
# Assuming your positive class is the second one in the 'class_names'
positive_class_probs = np.array(probs)[:, 1]
roc_auc = roc_auc_score(true_labels, positive_class_probs)

fpr, tpr, _ = roc_curve(true_labels, positive_class_probs)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


11. Visualize the test images along with the predictions.

In [ ]:
import matplotlib.pyplot as plt
import torchvision

def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [ ]:
def visualize_model(model, num_images=6):
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(test_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'true: {class_names[labels[j]]} pred: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    return


In [ ]:
visualize_model(model_ft, num_images=6)

12. Generate heatmaps to highlight regions of high influence to the predictions being made.

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Check if CUDA (GPU support) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#load model
model_ft = torch.load(PATH)
model_ft = model_ft.to(device)
model_ft.eval()

# Define preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

# Load and preprocess your medical image
image_path = '/content/gdrive/MyDrive/GasHisDB/GasHisDB_Final_Test/Abnormal/Abnormal-04038.png'
img = Image.open(image_path).convert('RGB')
input_tensor = preprocess(img)
input_batch = input_tensor.unsqueeze(0).to(device)  # Add a batch dimension and move to GPU

# Set up hooks for guided backpropagation
activation = []

def hook_fn(module, input, output):
    activation.append(output)

for module in model_ft.named_modules():
    if isinstance(module[1], torch.nn.ReLU):
        module[1].register_forward_hook(hook_fn)

# Forward pass
output = model_ft(input_batch)
prediction = output.argmax(1)

# Backpropagation to compute gradients
model_ft.zero_grad()
one_hot_output = torch.FloatTensor(1, output.size()[-1]).zero_().to(device)
one_hot_output[0][prediction] = 1
output.backward(gradient=one_hot_output)

# Get the gradients and compute the heatmap
if len(activation) > 0:
    gradients = activation[0][0].detach().cpu().numpy()
    heatmap = gradients.max(axis=0)

    # Apply thresholding to the heatmap
    threshold = 0.5  # Adjust this threshold value as needed
    heatmap[heatmap < threshold] = 0

    # Visualize the heatmap
    import matplotlib.pyplot as plt

    plt.imshow(heatmap, cmap='hot')
    plt.axis('off')
    plt.show()
else:
    print("No activations recorded.")


13. Test the model with a sample image patch.

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Check if CUDA (GPU support) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#load model
model_ft = torch.load(PATH)
model_ft = model_ft.to(device)
model_ft.eval()

# Define preprocessing transformations for prediction
data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and preprocess the image for prediction
image_path = '/content/gdrive/MyDrive/GasHisDB/GasHisDB_Final_Test/Abnormal/Abnormal-04038.png'
test_image = Image.open(image_path)
input_image = data_transforms(test_image).unsqueeze(0).to(device)  # Add a batch dimension

# Perform inference on the test image
with torch.no_grad():
    outputs = model(input_image)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Softmax to get probabilities
    _, predicted = torch.max(outputs, 1)

# Get the predicted class index and class name
predicted_class_index = predicted.item()
class_names = ['abnormal', 'normal']  # Replace with your class names

predicted_class_name = class_names[predicted_class_index]
predicted_probability = probabilities[0][predicted_class_index].item() * 100  # Probability in percentage

print(f"The predicted class is: {predicted_class_name}")
print(f"The probability of being {predicted_class_name} is: {predicted_probability:.2f}%")

# Set up hooks for guided backpropagation
activation = []

def hook_fn(module, input, output):
    activation.append(output)

for module in model_ft.named_modules():
    if isinstance(module[1], torch.nn.ReLU):
        module[1].register_forward_hook(hook_fn)

# Forward pass
output = model_ft(input_batch)
prediction = output.argmax(1)

# Backpropagation to compute gradients
model_ft.zero_grad()
one_hot_output = torch.FloatTensor(1, output.size()[-1]).zero_().to(device)
one_hot_output[0][prediction] = 1
output.backward(gradient=one_hot_output)

# Get the gradients and compute the heatmap
if len(activation) > 0:
    gradients = activation[0][0].detach().cpu().numpy()
    heatmap = gradients.max(axis=0)

    # Apply thresholding to the heatmap
    threshold = 0.5  # Adjust this threshold value as needed
    heatmap[heatmap < threshold] = 0

    # Visualize the heatmap
    # Descriptions of Red and Yellow regions in a heatmap
    print("Red Regions -  highest importance for the predicted class.")
    print("Yellow Regions - moderately high importance for the predicted class.")

    import matplotlib.pyplot as plt

    plt.imshow(heatmap, cmap='hot')
    plt.axis('off')
    plt.show()
else:
    print("No activations recorded.")

14. Test the model using a whole slide image from which the 120 by 120 patches were extracted.

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Check if CUDA (GPU support) is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#load model
model_ft = torch.load(PATH)
model_ft = model_ft.to(device)
model_ft.eval()

# Define preprocessing transformations for prediction
data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load and preprocess the image for prediction
image_path = '/content/gdrive/MyDrive/HCRF/test_dataset/cancer_108.png'
test_image = Image.open(image_path)
input_image = data_transforms(test_image).unsqueeze(0).to(device)  # Add a batch dimension

# Perform inference on the test image
with torch.no_grad():
    outputs = model(input_image)
    probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Softmax to get probabilities
    _, predicted = torch.max(outputs, 1)

# Get the predicted class index and class name
predicted_class_index = predicted.item()
class_names = ['abnormal', 'normal']  #  class names

predicted_class_name = class_names[predicted_class_index]
predicted_probability = probabilities[0][predicted_class_index].item() * 100  # Probability in percentage

print(f"The predicted class is: {predicted_class_name}")
print(f"The probability of being {predicted_class_name} is: {predicted_probability:.2f}%")

# Set up hooks for guided backpropagation
activation = []

def hook_fn(module, input, output):
    activation.append(output)

for module in model_ft.named_modules():
    if isinstance(module[1], torch.nn.ReLU):
        module[1].register_forward_hook(hook_fn)

# Forward pass
output = model_ft(input_image)
prediction = output.argmax(1)

# Backpropagation to compute gradients
model_ft.zero_grad()#to save memory
one_hot_output = torch.FloatTensor(1, output.size()[-1]).zero_().to(device)
one_hot_output[0][prediction] = 1
output.backward(gradient=one_hot_output)

# Get the gradients and compute the heatmap
if len(activation) > 0:
    gradients = activation[0][0].detach().cpu().numpy()
    heatmap = gradients.max(axis=0)

    # Apply thresholding to the heatmap
    threshold = 0.5  # Adjust this threshold value as needed
    heatmap[heatmap < threshold] = 0

    # Visualize the heatmap
    # Descriptions of Red and Yellow regions in a heatmap
    print("Red Regions -  highest importance for the predicted class.")
    print("Yellow Regions - moderately high importance for the predicted class.")

    import matplotlib.pyplot as plt

    plt.imshow(heatmap, cmap='hot')
    plt.axis('off')
    plt.show()
else:
    print("No activations recorded.")